In [ ]:
from myosuite.utils import gym
import mujoco
import mujoco.viewer
import numpy as np
import time


MyoSuite:> Registering Myo Envs


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/workspace/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/workspace/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_sp

In [ ]:
def poc_legal_lock():
    # 1. SETUP ENV
    env_name = "myoChallengeTableTennisP2-v0"
    print(f"Loading {env_name}...")
    env = gym.make(env_name)
    env.reset()
    
    sim = env.unwrapped.sim
    model = sim.model
    data = sim.data

    # =========================================================
    # PART A: FIND FINGER MUSCLES (The "Legal Glue" Setup)
    # =========================================================
    finger_keywords = ["thumb", "index", "middle", "ring", "little", "pinky", "pollux", "digiti"]
    finger_indices = []
    
    print("Scanning for finger actuators...")
    for i in range(model.nu):
        name = model.actuator_id2name(i)
        if name and any(k in name.lower() for k in finger_keywords):
            finger_indices.append(i)
            
    print(f"✅ Found {len(finger_indices)} finger muscles to LOCK.")

    # =========================================================
    # PART B: INJECT YOUR EXACT POSE (Frame 0 Only)
    # =========================================================
    print("Injecting your 'Golden Grip' coordinates at Frame 0...")
    
    perfect_fingers = {
        # THUMB
        "cmc_abduction": -0.334, "cmc_flexion": 0.0562, "mp_flexion": -0.511, "ip_flexion": -0.881,
        # INDEX
        "mcp2_flexion": 1.49, "mcp2_abduction": 0.147, "pm2_flexion": 1.3, "md2_flexion": 1.25,
        # MIDDLE
        "mcp3_flexion": 1.42, "mcp3_abduction": -0.0131, "pm3_flexion": 1.35, "md3_flexion": 1.04,
        # RING
        "mcp4_flexion": 1.48, "mcp4_abduction": -0.0681, "pm4_flexion": 1.36, "md4_flexion": 1.07,
        # PINKY (YOUR UPDATED VALUES)
        "mcp5_flexion": 1.39, "mcp5_abduction": -0.188, "pm5_flexion": 0.872, "md5_flexion": 1.57,
    }

    # Apply Position
    for name, val in perfect_fingers.items():
        try:
            jid = model.joint_name2id(name)
            adr = model.jnt_qposadr[jid]
            data.qpos[adr] = val
        except: pass

    # Forward physics to lock it in
    mujoco.mj_forward(model.ptr, data.ptr)

    # =========================================================
    # PART C: RUN PHYSICS WITH MUSCLE LOCK
    # =========================================================
    print("--------------------------------------------------")
    print("Running Simulation with MUSCLE LOCK (Action=1.0)...")
    print("The hand should stay closed purely by muscle strength.")
    
    with mujoco.viewer.launch_passive(model.ptr, data.ptr) as viewer:
        step = 0
        while viewer.is_running():
            
            # 1. Base Action: Random or Zero (Simulating the policy learning)
            action = env.action_space.shape
            
            # 2. *** THE LEGAL GLUE ***
            # Overwrite the finger indices to 1.0 (Max Flexion)
            action[finger_indices] = 1.0
            
            # 3. Step Environment
            env.step(action)
            viewer.sync()
            
            # 4. Check Drop Status
            if step % 20 == 0:
                p_err = env.unwrapped.obs_dict.get("palm_err", [0,0,0])
                dist = np.linalg.norm(p_err)
                status = "✅ HOLDING" if dist < 0.20 else "❌ DROPPED"
                print(f"Step {step}: Dist={dist:.4f}m | {status}")
            
            step += 1
            time.sleep(0.005)

if __name__ == "__main__":
    poc_legal_lock()

/workspace/myochallenge_2025eval/.venv/lib/python3.11/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    
action_space: Box(-1.0, 1.0, (275,), float32)
action_space.low/high: -1.0 1.0


In [13]:
_, _ = env.reset()

In [14]:
_, base_reward, terminated, truncated, info = env.step(env.action_space.sample())

In [ ]:
info['rwd_dict']

OrderedDict([('reach_dist', array(0.0368)),
             ('palm_dist', array(0.6068)),
             ('paddle_quat', array(0.3728)),
             ('torso_up', 0.9723172151249325),
             ('act_reg', array(-0.0159)),
             ('sparse', False),
             ('solved', array(False)),
             ('done', array(0)),
             ('dense', array(3.3259))])